In [ ]:
import importlib
import pandas as pd
import numpy as np
import os
import mysql.connector
from dotenv import load_dotenv
import constants
importlib.reload(constants)
from constants import DB_DRIVER_NAMES, DB_DRIVER_TO_API_NAME, API_RACE_NAMES
import pprint
import xgboost as xg
from sklearn.preprocessing import MinMaxScaler



In [2]:
df = pd.read_csv('all_race_data.csv')
print(df)

       year             race_name  lap_number       driver_id  position  \
0      2023    Bahrain Grand Prix           1  max_verstappen         1   
1      2023    Bahrain Grand Prix           1         leclerc         2   
2      2023    Bahrain Grand Prix           1           perez         3   
3      2023    Bahrain Grand Prix           1           sainz         4   
4      2023    Bahrain Grand Prix           1        hamilton         5   
...     ...                   ...         ...             ...       ...   
47743  2024  São Paulo Grand Prix          69           perez        11   
47744  2024  São Paulo Grand Prix          69         bearman        12   
47745  2024  São Paulo Grand Prix          69          bottas        13   
47746  2024  São Paulo Grand Prix          69          alonso        14   
47747  2024  São Paulo Grand Prix          69            zhou        15   

       lap_time  
0        99.019  
1       100.230  
2       101.063  
3       101.659  
4       1

In [3]:
#Finds the Lap time change between lap x and lap x + 1
#Removes outliers like pit stops and saftey cars/VSC
def get_lap_time_change(driver, year, race, stops):
    driver_race_2024_df = (df[(df.race_name == f"{race}") & (df.driver_id == f"{driver}") & (df.year == year)])
    if driver_race_2024_df.empty:
        return None
    total_time = 0
    time_changes = []
    prev_lap = driver_race_2024_df.iloc[0, 5]
    for index, row in driver_race_2024_df.iterrows():
        total_time += row['lap_time']
        time_change = prev_lap - row['lap_time']
        if (-9 < time_change < 6):
            time_changes.append(time_change)
        prev_lap = row['lap_time']
    avg_time_change = sum(time_changes) / (len(time_changes) + 1)
    return avg_time_change

In [ ]:

def api_name_to_db_name(race_name):
    race_name_map = {
        "Bahrain Grand Prix": "bahrain",
        "Saudi Arabian Grand Prix": "saudi-arabia",
        "Australian Grand Prix": "australia",
        "Japanese Grand Prix": "japan",
        "Chinese Grand Prix": "china",
        "Miami Grand Prix": "miami",
        "Emilia Romagna Grand Prix": "emilia-romagna",
        "Monaco Grand Prix": "monaco",
        "Canadian Grand Prix": "canada",
        "Spanish Grand Prix": "spain",
        "Austrian Grand Prix": "austria",
        "British Grand Prix": "great-britain",
        "Hungarian Grand Prix": "hungary",
        "Belgian Grand Prix": "belgium",
        "Dutch Grand Prix": "netherlands",
        "Italian Grand Prix": "italy",
        "Azerbaijan Grand Prix": "azerbaijan",
        "Singapore Grand Prix": "singapore",
        "United States Grand Prix": "united-states",
        "Mexico City Grand Prix": "mexico",
        "São Paulo Grand Prix": "sao-paulo",
        "Las Vegas Grand Prix": "las-vegas",
        "Qatar Grand Prix": "qatar",
        "Abu Dhabi Grand Prix": "abu-dhabi",
        "French Grand Prix" : "france"
    }
    return race_name_map.get(race_name, f"Error: '{race_name}' not found.")

In [5]:
load_dotenv()

db_host = os.getenv('DB_HOST')
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_name = os.getenv('DB_NAME')

In [6]:
#Finds the number of pit stops a driver completed per race
def find_number_pits(driver, year, race):
    connection = mysql.connector.connect(
        host=db_host,
        user =db_user,
        password = db_password,
        database = db_name,
        use_pure=True,
        ssl_disabled=True
    )

    mycursor = connection.cursor()

    mycursor.execute(f"select id from race where grand_prix_id = '{race}' and year = '{year}'")
    race_id_result = mycursor.fetchall()
    if len(race_id_result) == 0:
        return None
    race_id = race_id_result[0][0]
    mycursor.execute(f"select pit_stops from race_result where race_id = '{race_id}' and driver_id = '{driver}'")
    pit_stops_tuple = mycursor.fetchall()
    if len(pit_stops_tuple) == 0:
        return None
    pit_stops = pit_stops_tuple[0][0]
    return pit_stops


In [ ]:
#Finds the race data, including lap times and pit stops for each driver.
def get_race_pace_data():
    race_pace_data = {}
    for year in [2023, 2024]:
        for race in API_RACE_NAMES:
            race_data = []
            for driver in DB_DRIVER_NAMES:
                db_race_name = api_name_to_db_name(race)
                api_driver_name = DB_DRIVER_TO_API_NAME(driver)
                stops = find_number_pits(driver, year, db_race_name)
                avg_time_change = get_lap_time_change(api_driver_name, year, race, stops)
                if avg_time_change is not None:
                    race_data.append({
                        "driver": driver,
                        "avg_time_change": avg_time_change,
                        "stops": stops
                    })
            race_pace_data[f"{year} {race}"] = race_data
    return race_pace_data

In [8]:
total_data = get_race_pace_data()
pprint.pprint(total_data)

{'2023 Abu Dhabi Grand Prix': [{'avg_time_change': -0.0202452830188678,
                                'driver': 'carlos-sainz-jr',
                                'stops': 2},
                               {'avg_time_change': -0.0077115384615383895,
                                'driver': 'charles-leclerc',
                                'stops': 2},
                               {'avg_time_change': -0.06531372549019614,
                                'driver': 'alexander-albon',
                                'stops': 2},
                               {'avg_time_change': -0.049431372549019345,
                                'driver': 'daniel-ricciardo',
                                'stops': 2},
                               {'avg_time_change': 0.020019230769230435,
                                'driver': 'fernando-alonso',
                                'stops': 2},
                               {'avg_time_change': 0.0113653846153845,
                               

In [9]:
bahrain_2023 = total_data["2023 Bahrain Grand Prix"]
pprint.pprint(bahrain_2023)

[{'avg_time_change': -0.10817307692307693,
  'driver': 'carlos-sainz-jr',
  'stops': 2},
 {'avg_time_change': -0.10827777777777767,
  'driver': 'charles-leclerc',
  'stops': 2},
 {'avg_time_change': -0.18495999999999982,
  'driver': 'alexander-albon',
  'stops': 3},
 {'avg_time_change': -0.0683653846153842,
  'driver': 'fernando-alonso',
  'stops': 2},
 {'avg_time_change': -0.13846153846153825,
  'driver': 'george-russell',
  'stops': 2},
 {'avg_time_change': -0.16631250000000009, 'driver': 'guanyu-zhou', 'stops': 3},
 {'avg_time_change': -0.2817999999999998,
  'driver': 'kevin-magnussen',
  'stops': 3},
 {'avg_time_change': -0.08065384615384566,
  'driver': 'lance-stroll',
  'stops': 2},
 {'avg_time_change': -0.2494186046511633, 'driver': 'lando-norris', 'stops': 6},
 {'avg_time_change': -0.0969230769230765,
  'driver': 'lewis-hamilton',
  'stops': 2},
 {'avg_time_change': -0.2317551020408165,
  'driver': 'logan-sargeant',
  'stops': 3},
 {'avg_time_change': -0.10915686274509791,
  'd

In [10]:
default_keys = ['avg_time_change', 'driver', 'stops', 'race']
flattened_data = []

for race, data in total_data.items():
    if data:
        for entry in data:
            normalized_entry = {key: entry.get(key, None) for key in default_keys}
            normalized_entry['race'] = race
            flattened_data.append(normalized_entry)





race_data_df = pd.DataFrame(flattened_data)

print(race_data_df)

     avg_time_change           driver  stops                       race
0          -0.108173  carlos-sainz-jr    2.0    2023 Bahrain Grand Prix
1          -0.108278  charles-leclerc    2.0    2023 Bahrain Grand Prix
2          -0.184960  alexander-albon    3.0    2023 Bahrain Grand Prix
3          -0.068365  fernando-alonso    2.0    2023 Bahrain Grand Prix
4          -0.138462   george-russell    2.0    2023 Bahrain Grand Prix
..               ...              ...    ...                        ...
820        -0.126083     pierre-gasly    1.0  2024 São Paulo Grand Prix
821         0.129544     sergio-perez    2.0  2024 São Paulo Grand Prix
822        -0.011763  valtteri-bottas    1.0  2024 São Paulo Grand Prix
823         0.196186     yuki-tsunoda    2.0  2024 São Paulo Grand Prix
824         0.169259     esteban-ocon    1.0  2024 São Paulo Grand Prix

[825 rows x 4 columns]


In [11]:
race_data_df['performance'] = 1 /(race_data_df['avg_time_change'].abs() * race_data_df['stops'])
X = race_data_df[['avg_time_change', 'stops']]
y = race_data_df['performance']
race_data_df = race_data_df.dropna()


In [12]:
print(race_data_df.isnull().sum())

avg_time_change    0
driver             0
stops              0
race               0
performance        0
dtype: int64


In [ ]:
params = {
    'objective': 'reg:squarederror',
    'eval_metric': 'rmse',
    'eta': 0.1,
    'max_depth': 3
}
#Some are infinity so take that into account.
race_data_df['performance'].replace([float('inf'), float('-inf')], np.nan, inplace=True)
race_data_df.dropna(subset=['performance'], inplace=True)

xgb_model = xg.train(params, xg.DMatrix(race_data_df[['avg_time_change', 'stops']], label=race_data_df['performance']), num_boost_round=100)

race_data_df['predicted_efficiency'] = xgb_model.predict(xg.DMatrix(race_data_df[['avg_time_change', 'stops']]))

race_data_df['rank'] = race_data_df['predicted_efficiency'].rank(ascending=False)


sorted_df = race_data_df.sort_values(by='rank')
print(sorted_df[['driver', 'race', 'predicted_efficiency', 'rank']])

               driver                           race  predicted_efficiency  \
711   carlos-sainz-jr        2024 Italian Grand Prix           3656.013916   
773   fernando-alonso  2024 United States Grand Prix           3656.013916   
703    max-verstappen          2024 Dutch Grand Prix           1714.096680   
36       yuki-tsunoda  2023 Saudi Arabian Grand Prix           1714.096680   
800       liam-lawson    2024 Mexico City Grand Prix           1508.457520   
..                ...                            ...                   ...   
220    lewis-hamilton        2023 Belgian Grand Prix              0.392822   
753  daniel-ricciardo      2024 Singapore Grand Prix              0.392822   
482       guanyu-zhou       2024 Japanese Grand Prix              0.238669   
221    logan-sargeant        2023 Belgian Grand Prix              0.238669   
212   alexander-albon        2023 Belgian Grand Prix              0.238669   

      rank  
711    1.5  
773    1.5  
703    3.5  
36     3.5 

The next step is to average out the rankings across the races for each driver. 


In [ ]:
#Calculates and stores the driver tyre efficiency.
driver_efficiency = race_data_df.groupby('driver')['predicted_efficiency'].mean().reset_index()
driver_efficiency.rename(columns={'predicted_efficiency': 'efficiency_score'}, inplace=True)
scaler = MinMaxScaler(feature_range=(1.0, 1.09))
driver_efficiency['efficiency_score'] = scaler.fit_transform(driver_efficiency[['efficiency_score']])
driver_efficiency['efficiency_score'] = 2.09 - driver_efficiency['efficiency_score']
print(driver_efficiency)
%store driver_efficiency

              driver  efficiency_score
0    alexander-albon          1.089638
1    carlos-sainz-jr          1.039664
2    charles-leclerc          1.085786
3   daniel-ricciardo          1.087017
4       esteban-ocon          1.088308
5    fernando-alonso          1.045747
6   franco-colapinto          1.087274
7     george-russell          1.089411
8        guanyu-zhou          1.090000
9    kevin-magnussen          1.089999
10      lance-stroll          1.086351
11      lando-norris          1.086976
12    lewis-hamilton          1.087382
13       liam-lawson          1.000000
14    logan-sargeant          1.089939
15    max-verstappen          1.067213
16   nico-hulkenberg          1.087153
17     oscar-piastri          1.086413
18      pierre-gasly          1.088400
19      sergio-perez          1.088617
20   valtteri-bottas          1.088266
21      yuki-tsunoda          1.068548
Stored 'driver_efficiency' (DataFrame)


/usr/lib/python3/dist-packages/IPython/extensions/storemagic.py:229: UserWarning: using autorestore/driver_efficiency requires you to install the `pickleshare` library.
  db[ 'autorestore/' + arg ] = obj


Some of these findings did spark some concern as it denotes that Liam Lawson is incredible at tyre management. This is likely due to the lack of data as he only competed in a few races within the dataset. Other than this outlier the rest of the values seem acceptable and so as long as the results are skewed these will be used.

In [15]:
driver_efficiency.to_csv('driver_efficiency.csv', index=False)